In [87]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)
import math
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import cross_val_score

In [88]:
import missingno
import ml_utils

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [89]:

name = "Navneet Kaur"

In [90]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
174,1,0.774,0.093,0.479,0.232,0.432,0.892,0.021,0.041,0.944,...,0.957,0.171,0.236,0.361,0.717,0.388,0.351,0.973,0.258,0.888
68,1,0.850,0.576,0.899,0.360,0.879,0.055,0.010,0.637,0.471,...,0.228,0.577,0.366,0.930,0.132,0.782,0.248,0.038,0.792,0.099
75,0,0.125,0.464,0.599,0.673,0.205,0.784,0.210,0.378,0.723,...,0.722,0.577,0.419,0.109,0.083,0.966,0.088,0.849,0.870,0.829
120,1,0.390,0.208,0.249,0.579,0.419,0.844,0.419,0.084,0.109,...,0.620,0.263,0.722,0.614,0.825,0.472,0.877,0.059,0.005,0.633
124,0,0.244,0.030,0.627,0.907,0.047,0.127,0.073,0.615,0.032,...,0.730,0.364,0.125,0.291,0.138,0.494,0.921,0.595,0.399,0.445


### Starting

For this assignment, you have a small training set, so combatting overfitting is key in being accurate!

In [91]:
df.shape

(250, 201)

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Columns: 201 entries, target to var_200
dtypes: float64(200), int64(1)
memory usage: 392.7 KB


In [93]:
df_eda = ml_utils.edaDF(df,"target")
print(df_eda.giveTarget())

target


<li>Eda stuff- it takes long to complete,so i decide not to show here,but no missing or duplicated values were found during analysis.

#### Do Modelling Stuff

Make a tree model (of some vareity) and make it fit well. Keep in mind the possibility of your tree overfitting, and think of steps you may need to combat that shoudl it occur. 

In [94]:
y = np.array(df["target"]).reshape(-1,1).ravel()
X = np.array(df.drop(columns={"target"}))
X_train, X_test, y_train, y_test = train_test_split(X, y) #spliiting in test training set

In [99]:
# Create the decision tree classifier-tested with both tree classifiers decision tree and random forest 
#tree_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
# Create the pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', rf_clf)])
# Define the parameter grid
param_grid = { 'classifier__max_depth': [3, 5, 7, 9],
    'classifier__min_samples_split': [4, 6, 8, 10],
    'classifier__min_samples_leaf': [ 5, 7, 9],
    #'classifier__max_features': ['auto', 'sqrt', 'log2']
    'classifier__criterion': [ 'entropy'],
    #'classifier__n_estimators': [ 50, 100,200]
    }
# Create the grid search object
grid_search = GridSearchCV(pipe, param_grid, cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters: ", grid_search.best_params_)
print("Best Training score: ", grid_search.best_score_)


Best parameters:  {'classifier__criterion': 'entropy', 'classifier__max_depth': 7, 'classifier__min_samples_leaf': 5, 'classifier__min_samples_split': 8}
Best Training score:  0.6580369843527738


In [100]:
best = grid_search.best_estimator_
print("Model test Score",best.score(X_test, y_test))
print("Final Model", best)

Model test Score 0.6190476190476191
Final Model Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(criterion='entropy', max_depth=7,
                                        min_samples_leaf=5,
                                        min_samples_split=8))])


### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [ ]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<li>with decision tree classifier & the Best parameters - Best score were  0.56& for testing data it was 0.38 as there is large difference in training and testing accuracy so our model seems to overfitted.So,Random forest classifier was used.
<li>i used CROSS VALIDATION to combat overfitting 
<li>if we decided to test more parameters in grid search cv it would take a very long time to run so i decided to test few parameters at moment to find best to increase accuracy/limit overfitting.
<li>on changing  tree to random forest & choosing three  Best parameters
Best score:  0.6368421052631579 & for test-0.52,
<li>on adding max feature parameter, feature parameter is log2 accuracy changed to 0.63 and for testing -.50 #but it takes about 5 mins to run
<li>accuraccy increased to 0.71 if i specified entropy.
<li>After trying many combination of parameters, Finally best model has these  parameters:  {'classifier__criterion': 'entropy', 'classifier__max_depth': 5, 'classifier__min_samples_leaf': 7, 'classifier__min_samples_split': 4} and accuracy of 0.66 and 0.62 with training and testing data respectively .Moreover (train_acc - test_acc) < 0.05,so our best model is not overfitted.
    